In [1]:
import os

path = 'C:/Users/'+os.getlogin()+'/Google Drive/University/Dissertation'
#datapath = 'C:/Users/'+os.getlogin()+'/Dissertation Data'
datapath = 'E:/Dissertation Data'

os.chdir(path+'/Code')
os.getcwd()

'C:\\Users\\User\\Google Drive\\University\\Dissertation\\Code'

In [2]:
import pickle
import pandas as pd
import re
import numpy as np

import nltk
from nltk.tokenize import MWETokenizer

from glove import Corpus, Glove

from gensim.models import Word2Vec

In [3]:
# On 10% English wiki, POS tagged

sf = open(datapath+'/Corpora/wiki/enwiki_20200520/Tagged/enwiki_20200520_10pc_tagged_clean.txt', 'r', encoding='utf-8')

for lines in range(5):
    print(sf.readline())

Autism|NN is|VBZ a|DT developmental|JJ disorder|NN characterized|VBN by|IN difficulties|NNS with|IN social|JJ interaction|NN and|CC communication|NN and|CC by|IN restricted|JJ and|CC repetitive|JJ behavior|NN 



Parents|NNS often|RB notice|VBP signs|NNS during|IN the|DT first|JJ three|CD years|NNS of|IN their|PRP$ child|NN's life|NN 



These|DT signs|NNS often|RB develop|VBP gradually|RB though|IN some|DT children|NNS with|IN autism|NN experience|NN worsening|VBG in|IN their|PRP$ communication|NN and|CC social|JJ skills|NNS after|IN reaching|VBG developmental|JJ milestones|NNS at|IN a|DT normal|JJ pace|NN 



In [4]:
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

from nltk.tokenize import WhitespaceTokenizer

from nltk.corpus.reader.util import read_line_block

In [5]:
w10p_t = PlaintextCorpusReader(datapath+'/Corpora/wiki/enwiki_20200520/Tagged','enwiki_20200520_10pc_tagged_clean.txt',
                            word_tokenizer = WhitespaceTokenizer(),
                            para_block_reader=read_line_block
                            )

In [6]:
# Import word and sentence generators

from generators import sent_gen, word_gen, Sent_Seq

In [7]:
# Collate n-grams

from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder

from nltk.metrics import (
    BigramAssocMeasures,
    TrigramAssocMeasures,
    NgramAssocMeasures,
)

from nltk.metrics.spearman import (
    spearman_correlation,
    ranks_from_scores,
)

In [8]:
# Stopwords from corpus - 50 most frequent

with open(datapath+'/Corpora/wiki/enwiki_20200520/Tagged/stop_clean.pkl', 'rb') as pfile:
    stop = pickle.load(pfile)

In [9]:
stop

{'0|CD',
 '1|CD',
 '2|CD',
 'A|DT',
 'He|PRP',
 'In|IN',
 'It|PRP',
 'New|NNP',
 'The|DT',
 'after|IN',
 'also|RB',
 'and|CC',
 'an|DT',
 'are|VBP',
 'as|IN',
 'at|IN',
 'a|DT',
 'been|VBN',
 'be|VB',
 'but|CC',
 'by|IN',
 'first|JJ',
 'for|IN',
 'from|IN',
 'had|VBD',
 'has|VBZ',
 'he|PRP',
 'his|PRP$',
 'in|IN',
 'is|VBZ',
 'its|PRP$',
 'it|PRP',
 'not|RB',
 'of|IN',
 'one|CD',
 'on|IN',
 'or|CC',
 'that|IN',
 'that|WDT',
 'their|PRP$',
 'the|DT',
 'this|DT',
 'to|IN',
 'to|TO',
 'two|CD',
 'was|VBD',
 'were|VBD',
 'which|WDT',
 'who|WP',
 'with|IN'}

In [10]:
min_freq = 20
eval_count = 500000

In [11]:
ngram_eval = pd.read_pickle(datapath+'/Corpora/wiki/enwiki_20200520/Tagged/ngram_eval_nopn.pkl')

ngram_eval

,ngram,freq,poisson,len,batch
0,"(MSC1|NN, MSC2|NN)",20,-666.206003,2,1
1,"(resting_place_coordinates|NNS, burial_place|VBP)",20,-666.206003,2,1
2,"(burial_place|VBP, burial_coordinates|NNS)",20,-670.238680,2,2
3,"(resting_place|NN, resting_place_coordinates|NNS)",20,-673.776236,2,2
4,"(Gm3|NN, ∕|SYM)",20,-677.905253,2,1
...,...,...,...,...,...
499995,"(and|CC, bit|NN)",31,-1837.108921,2,1
499996,"(also|RB, the|DT, day|NN)",22,-1837.109386,3,2
499997,"(public|NN, on|IN, a|DT)",23,-1837.110118,3,4
499998,"(coast|NN, during|IN, the|DT)",23,-1837.117093,3,7


In [12]:
batch_count = max(ngram_eval.batch)

In [13]:
ngram_eval.batch.value_counts()

 1     171283
 2     122965
 3      76805
 4      51536
 5      30032
 6      19160
 7      12228
 8       6916
 9       3513
-1       3212
 10      2013
-2        337
Name: batch, dtype: int64

In [14]:
from nltk.tokenize import MWETokenizer
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Flatten down to a single number
def cosim(x,y):
    return cosine_similarity(x.reshape(1,-1), y.reshape(1,-1))[0][0]

In [15]:
for s in Sent_Seq(w10p_t, remchars = "[`¬@^\"]"):
    print(s)
    break

['Autism|NN', 'is|VBZ', 'a|DT', 'developmental|JJ', 'disorder|NN', 'characterized|VBN', 'by|IN', 'difficulties|NNS', 'with|IN', 'social|JJ', 'interaction|NN', 'and|CC', 'communication|NN', 'and|CC', 'by|IN', 'restricted|JJ', 'and|CC', 'repetitive|JJ', 'behavior|NN']


In [16]:
%%time

batch_dfs = {}

for bb in range(batch_count):
    print('Processing batch {} of {}'.format(bb+1,batch_count))
    
    # Subset DataFrame
    batch_dfs[bb] = ngram_eval[ngram_eval.batch == bb+1].reset_index(drop=True)
    
    # Initialise MWETokenizer
    batch_token_mwe = MWETokenizer(list(batch_dfs[bb].ngram) , separator='+')
    
    # Build model
    print('Building word2vec model')
    sents_mwe = Sent_Seq(w10p_t, tokenizer = batch_token_mwe, remchars = "[`¬@^\"]")
    
    print(' Training model')
    batch_model = Word2Vec(sents_mwe,
                             min_count = 20,  # 20 matches R&E on EN Wiki
                             size = 400,
                             workers = 8,
                             window = 5,
                             sg = 0,         # CBOW
                             sample = 10e-5, # Subsampling
                             negative = 10
                            )

    # Save model
    print(' Saving model')
    batch_model.save(datapath+'/Models/1 w2v/Tagged/w10p_tagged_nopn_batch{}.model'.format(bb+1))
    


Processing batch 1 of 10
Building word2vec model
 Training model
 Saving model
Processing batch 2 of 10
Building word2vec model
 Training model
 Saving model
Processing batch 3 of 10
Building word2vec model
 Training model
 Saving model
Processing batch 4 of 10
Building word2vec model
 Training model
 Saving model
Processing batch 5 of 10
Building word2vec model
 Training model
 Saving model
Processing batch 6 of 10
Building word2vec model
 Training model
 Saving model
Processing batch 7 of 10
Building word2vec model
 Training model
 Saving model
Processing batch 8 of 10
Building word2vec model
 Training model
 Saving model
Processing batch 9 of 10
Building word2vec model
 Training model
 Saving model
Processing batch 10 of 10
Building word2vec model
 Training model
 Saving model
Wall time: 2d 11h 27min 1s
